In [1]:
import sys
sys.path.append( ".." )
from pathlib import Path
from sqlalchemy import create_engine, inspect
import pandas as pd

from scripts.s3Utils import decryptCredentials, initialize

In [ ]:
# Create other cell and delete afterwards
credKey = ""

In [3]:
credPath =  Path( "../data/credentials.enc").resolve()
credentialsDict = decryptCredentials( credKey, credPath)

Object Store Names:['XChem', 'MinIO']
Credentials Associated with each Object Store: ['endpoint_url', 'access_key', 'secret_key']


In [15]:
client = initialize( "XChem", cred_dict=credentialsDict)

print( "Object Store Ping Response: {}".format( client.list_buckets()["ResponseMetadata"]["HTTPStatusCode"] ) )


key = 'data/2015/lb13320-1/processing/database/soakDBDataFile.sqlite' # 'data/2015/lb13320-1/processing/database/DataFileBackups/soakDBDataFile.sqlite' # r"data/2015/lb13308-1/processing/database/soakDBDataFile.sqlite\n"
bucket = "xchem"

storePath = Path( r"../data/soakDBDataFile.sqlite").resolve()
print(storePath)

client.download_file( bucket, key, storePath )
print( "Managed to download file")

Object Store Ping Response: 200
D:\OneDrive - University of Edinburgh\Documents\000-PhDCodeSpace\XAIDA\XAIDA\data\soakDBDataFile.sqlite
Managed to download file


In [19]:
from sqlalchemy import create_engine, inspect
import pandas as pd
from pathlib import Path

# Path to your .sqlite file

sqliteFilePath = Path( "../data/soakDBDataFile.sqlite").resolve()

print("Sqlite File Path: {}".format( sqliteFilePath ) )
# Step 1: Create an SQLAlchemy engine to connect to the SQLite database
engine = create_engine(f"sqlite:///{sqliteFilePath}")

# Step 2: Inspect the database to see available tables
inspector = inspect(engine)
tables = inspector.get_table_names()
print("Tables in the database:")
print(tables)

# Step 3: Load a specific table into a pandas DataFrame
tables = ['depositTable', 'mainTable', 'panddaTable', 'soakDB']# ['mainTable', 'panddaTable', 'collectionTable', 'depositTable','soakDB', 'zenodoTable','Pucks', ]
loadtable = lambda table_name : pd.read_sql(f"SELECT * FROM {table_name}", con=engine)
tabledf_dict = {table_name : loadtable(table_name) for table_name in tables[:4]  }

# for df in tabledf_dict.values():
#     filter_colms = [ colm for colm in df.columns if not all( df[colm].isna() ) ]    # Show columns with data
#     display( filter_colms)
    # display( df] )
# tabledf_lst = maindf, panddadf, collectdf, depoistdf 
# print(type( tabledf_dict.values()))

# display( list( tabledf_dict.values() )[0].iloc[:,:3])



# Step 6: Close the connection (optional, as SQLAlchemy handles it automatically)
engine.dispose()

Sqlite File Path: D:\OneDrive - University of Edinburgh\Documents\000-PhDCodeSpace\XAIDA\XAIDA\data\soakDBDataFile.sqlite
Tables in the database:
['depositTable', 'mainTable', 'panddaTable', 'soakDB']


In [23]:
display( tabledf_dict["mainTable"].head() )

,ID,LabVisit,LibraryPlate,SourceWell,LibraryName,CompoundSMILES,CompoundCode,CrystalPlate,CrystalWell,EchoX,...,DatePanDDAModelCreated,RefinementCIFStatus,RefinementCIFprogram,RefinementStatus,Deposition_PDB_ID,Deposition_Date,RefinementProgram,Deposition_PDB_file,Deposition_mmCIF_model_file,Deposition_mmCIF_SF_file
0,1,lb13320-1,DSPL,A01,DSPL,COc1cc(ccc1N)N2CCOCC2,N13417a,441330002553,A01d,-179.2228,...,None,restraints generated,acedrg,pending,None,None,None,None,None,None
1,2,lb13320-1,DSPL,A02,DSPL,CC[NH+]1CCCCC1,N13353a,441330002553,B01a,399.2647,...,None,None,None,None,None,None,None,None,None,None
2,3,lb13320-1,DSPL,A03,DSPL,CC(=O)Nc1nnc(s1)COC,N13517a,441330002553,B01c,-504.7559,...,None,restraints generated,acedrg,pending,None,None,None,None,None,None
3,4,lb13320-1,DSPL,A04,DSPL,CCNC(=O)Nc1cc(on1)C,N13570a,441330002553,E01c,-673.7883,...,None,restraints generated,acedrg,pending,None,None,None,None,None,None
4,5,lb13320-1,DSPL,A05,DSPL,CC(C)(C)OC(=O)N1CC[NH+](CC1)CCO,N13419a,441330002553,E01d,-219.6951,...,None,restraints generated,acedrg,pending,None,None,None,None,None,None


# Intro

List of tables:

- Main Table
    - Contains most of the overall data for all the steps of the pipeline
- Collection Table
    - Contains information about each autoprocessing protocol used for each diffraction pattern data, including quality metics, file paths, parameters used...
- PanDDA Table
    - Conatins information about the pandda processing of the autoprocessed data
- depositTable
- soakDB
- zenodoTable
- pucks

## | **Main Table** |

In [11]:
main_df = tabledf_dict["mainTable"]
pandda_df = tabledf_dict["panddaTable"]
collect_df = tabledf_dict["collectionTable"]

### Key Metadata

<ins> Lab Visit and Lab Collection Visit </ins>

The lab visit corresponds to the visit that exists in the dls record as visit. Then, there can be visits happening latter to collect the data, but the data gets stored in the original folder.

Columns: LabVisit DataCollectionVisit

<ins> Experiment Type </ins>

There are two columns: LibraryPlate and LibraryName

NOTE: There are cases when labels miss on one, but not on the other.

# **Q.**
How do I tell whether it is a solvent experiment, a fragment screen, or anything else? The labels seem very arbitrary.

In [34]:
# Q: Is there a preset list of names that these can take, or are they selected separately?

# crys_libplate = main_df["CrystalName"][main_df["LibraryPlate"] == "done"]
# crys_libname =  main_df["CrystalName"][main_df["LibraryName"] == "done"]


print( "Library Plates Labels: ", list( set( main_df[ "LibraryPlate" ] )))
print( "Library Names Labels: ", list( set( main_df[ "LibraryName" ] )))

# print("Number of crystals which were soaked: ", len( crys_soakstatus ) )
# print("Number of crystals which were soaked: ", len( crys_soakstatus ) )

Library Plates Labels:  ['SpotXplorer', '1851376-Y4-02', '1530852-Y4-89', '1530852-Y4-51', None, '1530852-Y4-83', 'Probing_Fragment_all', 'All-probing', 'Levente Kollar-Ivan Ahel', '1530852-Y4-163', '1530852-Y4-288', '1851376-Y4-01', 'CHICKV-xp02', '1530852-Y4-112', 'CHICKV-xp01', "Peter O'Brien Compounds", 'DSIPoised', 'EnamineEF', '1530852-Y4-62', '1530852-Y4-134', '1530852-Y4-16', '1530852-Y4-39', 'Solvent Test', 'DSIPoised_DMSO', '1530852-Y4-65']
Library Names Labels:  ['EnamineEssentialFragments', 'SpotXplorer', None, '1851376-Y4-01', '1851376-Y4-02', 'York 3D', 'Minifrags', 'ASAP-Mac1', 'Levente Kollar-Ivan Ahel', "Peter O'Brien Compounds", 'CHICKV-xp02', 'PepLite', 'CHICKV-xp01', 'Solvent', 'DSIPoised', 'Fraglite', 'Diffraction Test']


<ins>Compound Information</ins>

- Unique Identifiers: "CrystalName"

Notes: 

- the compound code includes solvent information

- There might be a repeat of the same compound being used. This can mean that it is used for different targets (which is unusual given that the norm is that each visit uses one target protein), or duplicates, or something else.

In [82]:
compound_info = main_df[ ["CrystalName", "CompoundCode", "CompoundSMILES"] ]
# display(compound_info[300:305])
print("Total number of unique compounds codes: ", len(list( set( main_df["CompoundCode"][  main_df["CompoundCode"].isna() == False ] ) ) ) )
print("Total number of unique compounds SMILES: ", len(list( set( main_df["CompoundSMILES"][  main_df["CompoundSMILES"].isna() == False ] ) ) ) )
print("Total number of compound SMILES: ", len( main_df["CompoundSMILES"].dropna() ) )



Total number of unique compounds codes:  1655
Total number of unique compounds SMILES:  1635
Total number of compound SMILES:  3466


<ins> Protein Information <ins>
- Protein Name: "ProteinName"


In [50]:
print("Total number of unique proteins: ", len(list( set( main_df["ProteinName"][  main_df["ProteinName"].isna() == False ] ) ) ) )

Total number of unique proteins:  2


<ins> Soaking </ins>

In [30]:
crys_soakstatus = main_df["CrystalName"][main_df["SoakStatus"] == "done"] # ??? what is exported ???7
print( "Soak Status Labels: ", list( set( main_df[ "SoakStatus" ] )))
print("Number of crystals which were soaked: ", len( crys_soakstatus ) )

Soak Status Labels:  [None, 'exported', 'done']
Number of crystals which were soaked:  3645


<ins> Harvest </ins>

Exported? -> Maybe they were moved elsewhere?

In [33]:
crys_harvested = main_df["CrystalName"][main_df[ "HarvestStatus" ] == "done"] 
crys_failed = main_df["CrystalName"][main_df[ "HarvestStatus" ] == "fail"] 
print( "Harvest Status Labels: ", list( set( main_df[ "HarvestStatus" ] )))
print("Number of crystals that got harvested: ",len( crys_harvested ) )
print("Number of crystals that failed harvest: ", len( crys_failed) )

Harvest Status Labels:  [None, 'exported', 'done', 'fail']
Number of crystals that got harvested:  2926
Number of crystals that failed harvest:  413


<ins>Mounting</ins>

Check the success of mounting. If there was failing in harvesting or soaking, this would show fail.

In [58]:
bool_lst = [ val[:2] == "OK" if val != None else False for val in main_df[ "MountingResult" ] ]
print( "Mounting Status Labels: ", list( set( main_df[ "MountingResult" ] )))
crys_mounted = main_df["CrystalName"][bool_lst]
print("Number of crystals that got mounted: ", len( crys_mounted ) )


Mounting Status Labels:  [None, 'OK: Crystalline:No comment', 'FAIL: No comment:Cracked', 'OK: Precipitated:Dissolved', 'OK: No comment:Cracked', 'OK: No comment:No comment', 'FAIL: No comment:Coloured', 'FAIL: No comment:Dissolved', 'FAIL: No comment:Melted', 'OK: Precipitated:No comment', 'OK: No comment:Dissolved', 'OK: No comment:Jelly', 'OK: No comment:Coloured', 'FAIL: Bad Dispense:No comment', 'FAIL: No comment:No comment']
Number of crystals that got mounted:  2826


<ins>IspyB DB</ins>

Some were NOT soaked and NOT Mounted, but were harvested and exported to ispyB -> Probably were crystallized here without being soaked and got diffracted elsewhere but got exported into ispyb?


In [80]:
uniq_lst =list( set( [value[:17] if type(value) == str else value if value == None else value for value in main_df[ "ispybStatus" ]] ))
bool_lst = [ value[:17] == 'exported to ispyb' if type(value) == str else False for value in main_df[ "ispybStatus" ] ]
print( "Database export Status Labels: ", uniq_lst )
crys_inispyb = main_df["CrystalName"][bool_lst]
print("Number of crystals that got exported to IspyB: ",len( crys_inispyb ) )

# Trying to identify the origin of the ones that got export but not mounted. The number of exported == number of harvested
not_mounted_1 = set( crys_inispyb)  - set( crys_mounted )
print(len(not_mounted_1)) #
# print(len()) 
print(sorted(list( not_mounted_1 ) ) ) 

print("All the crystals harvested correspond to all the crystals exported to ispyb") if set(crys_inispyb) == set(crys_harvested) else print("Not all crystals harvested were exported to ispyb") 

Database export Status Labels:  [None, 'exported to ispyb']
Number of crystals that got exported to IspyB:  2926
100
['CHIKV_MacA-x0001', 'CHIKV_MacA-x0002', 'CHIKV_MacA-x0003', 'CHIKV_MacA-x0004', 'CHIKV_MacA-x0005', 'CHIKV_MacA-x0006', 'CHIKV_MacA-x0007', 'CHIKV_MacA-x0008', 'CHIKV_MacA-x0009', 'CHIKV_MacA-x0010', 'CHIKV_MacA-x0011', 'CHIKV_MacA-x0012', 'CHIKV_MacA-x0013', 'CHIKV_MacA-x0014', 'CHIKV_MacA-x0015', 'CHIKV_MacA-x0016', 'CHIKV_MacA-x0017', 'CHIKV_MacA-x0018', 'CHIKV_MacA-x0019', 'CHIKV_MacA-x0020', 'CHIKV_MacA-x0021', 'CHIKV_MacA-x0022', 'CHIKV_MacA-x0023', 'CHIKV_MacA-x0024', 'CHIKV_MacA-x0025', 'CHIKV_MacA-x0026', 'CHIKV_MacA-x0027', 'CHIKV_MacA-x0028', 'CHIKV_MacA-x0029', 'CHIKV_MacA-x0030', 'CHIKV_MacA-x0031', 'CHIKV_MacA-x0032', 'CHIKV_MacB-x0032', 'CHIKV_MacB-x0033', 'CHIKV_MacB-x0034', 'CHIKV_MacB-x0035', 'CHIKV_MacB-x0036', 'CHIKV_MacB-x0037', 'CHIKV_MacB-x0038', 'CHIKV_MacB-x0039', 'CHIKV_MacB-x0040', 'CHIKV_MacB-x0041', 'CHIKV_MacB-x0042', 'CHIKV_MacB-x0043', 'C

<ins>Data collection </ins>

- Data Collection Outcome (success or Failed ): "DataCollectionOutcome"
-  


In [86]:
crys_datacollected = main_df["CrystalName"][main_df[ "DataCollectionOutcome" ] == "success"] 
crys_datacollected_fail = main_df["CrystalName"][main_df[ "DataCollectionOutcome" ] == "Failed - low resolution"] 
print( "Collection Status Labels: ", list( set( main_df[ "DataCollectionOutcome" ] )))
print( "Number of crystals which data got collected: ", len(crys_datacollected))
print( "Number of crystals which failed data collection: ", len(crys_datacollected_fail ) )
# print( len(crys_mounted)- len(crys_datacollected_fail) )

Collection Status Labels:  [None, 'Failed - low resolution', 'success']
Number of crystals which data got collected:  2540
Number of crystals which failed data collection:  6


<ins>Autoprocessing</ins>

Column name for Path to autoprocessed file: DataProcessingPathToMTZfile

In [13]:
print( "Path of autoprocessed file used: ",list( main_df["DataProcessingPathToMTZfile"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )

Path of autoprocessed file used:  /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/autoprocessing/lb32633-19-CHIKV_MacB-x0270_1xia2-dials_b12ee8bc-44d0-47ee-930a-5378514db6b5/lb32633v19_xCHIKVMacBx02701_free.mtz


<ins>Dimple </ins>

- Path to dimple output pdb file: DimplePathToPDB
- Path to dimpple output mtz file: DimplePathToMTZ
- Path to dimple reference model file: DimpleReferencePDB
- Dimple Status: DimpleStatus
- Dimple Metrics: DimpleRcryst | DimpleRfree

Note that the dimple reference model gets updated with the PanDDA 

In [17]:
crys_dimplesuccess = main_df["CrystalName"][ main_df[ "DataProcessingDimpleSuccessful" ].isin( ["TRUE","True"] ) ] 
print( "Dimple pdb Path: \t", list( main_df["DimplePathToPDB"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Dimple mtz Path: \t", list( main_df["DimplePathToMTZ"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print("Reference Path: \t" , list( main_df["DimpleReferencePDB"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print("Dimple Status: \t\t" , list( main_df["DimpleStatus"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print("Dimple Metrics: \t Dimple Rcryst =" , list( main_df["DimpleRcryst"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0], 
      "| Dimple Rfree =", list( main_df["DimpleRfree"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0] , "\n" )

print( "Dimple Status Labels: ", list( set( main_df[ "DimpleStatus" ] ) ) )
print( "Dimple Processing Status Labels: ", list( set( main_df[ "DataProcessingDimpleSuccessful" ] ) ) )
print( "Number of crystals which data got collected: ", len(crys_dimplesuccess))



Dimple pdb Path: 	 /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/dimple.pdb
Dimple mtz Path: 	 /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/dimple.mtz
Reference Path: 	 /dls/labxchem/data/lb32633/lb32633-6/processing/reference/CHIKV_MacB-x0257-ground-state.pdb
Dimple Status: 		 finished
Dimple Metrics: 	 Dimple Rcryst = 0.19019 | Dimple Rfree = 0.2166 

Dimple Status Labels:  [None, 'running', 'finished']
Dimple Processing Status Labels:  [None, 'True', 'TRUE']
Number of crystals which data got collected:  2387


<ins> PanDDAS Processing </ins>

- CrystalName : Unique Identifier for other tables

- DimplePANDDAwasRun: Was Panddas run on crystal data?

- DimplePANDDAreject: Was crystal data rejected by Panddas?

- DimplePANDDAhit: Did Panddas identify a hit?

- DimplePANDDApath: Path 



In [19]:
crys_panddarun = main_df["CrystalName"][main_df["DimplePANDDAwasRun"] == "TRUE"]
crys_panddaaccept =  main_df["CrystalName"][main_df["DimplePANDDAreject"] == "FALSE"]
crys_panddahit = main_df["CrystalName"][main_df["DimplePANDDAhit"] == "TRUE"]

print( "PanDDa Run Status Labels: ", list( set( main_df[ "DataCollectionOutcome" ] )))
print( "PanDDa Status Labels: ", list( set( main_df[ "PANDDAStatus" ] ) ) )
print( "PanDDa reject Status Labels: ", list( set( main_df[ "DimplePANDDAreject" ] )))
print( "PanDDa hit Status Labels: ", list( set( main_df[ "DimplePANDDAhit" ] )))
print( "Path of location for PanDDa processing:", list( main_df["DimplePANDDApath"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )

print( "\nNumber of crystal run through PanDDas: ", len(crys_panddarun) )
print("Number of crystals data accepted by PanDDas: ", len(crys_panddaaccept))      
print("\nNumber of crystals with identified hits by PanDDAS: ",len(crys_panddahit) )
print("Number of crystals without identified hits by PanDDAS: ", len(crys_panddaaccept)- len(crys_panddahit) )
print("Percentage of crystals accepted by Panddas with identified hits: ", round( len(crys_panddahit)/  len(crys_panddaaccept),4 )*100 , "%" )


PanDDa Run Status Labels:  [None, 'success', 'Failed - low resolution']
PanDDa Status Labels:  [None]
PanDDa reject Status Labels:  [None, 'FALSE']
PanDDa hit Status Labels:  [None, 'TRUE']
Path of location for PanDDa processing: /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/panddas/20240906_pandda2_2472-2588

Number of crystal run through PanDDas:  2142
Number of crystals data accepted by PanDDas:  2142

Number of crystals with identified hits by PanDDAS:  832
Number of crystals without identified hits by PanDDAS:  1310
Percentage of crystals accepted by Panddas with identified hits:  38.84 %


<ins>Refinement</ins>

### Q.

What is the difference between refinement outcome and status?


Metrics:

- RefinementResolution, RefinementResolutionTL
- RefinementRcryst, RefinementRcrystTraficLight, RefinementRfree, RefinementRfreeTraficLight
- RefinementLigandCC
- RefinementRmsdBonds, RefinementRmsdBondsTL
- RefinementRmsdAngles, RefinementRmsdAnglesTL 

Status:
- Refinement Outcome Labels: RefinementOutcome
- Refinement Status Labels: RefinementStatus
- Refinement Status of Compound: RefinementCIFStatus

Paths:
- Path for refined Compound: RefinementCIF
- Path of most refined model: RefinementPDB_latest
- Path of most refined mtz: RefinementMTZ_latest
- Path of most refined mmcif: RefinementMMCIFmodel_latest
- Path of most refined mmcif constrains: RefinementMMCIFreflections_latest
- Path to refined bound conformatoin: RefinementBoundConformation



Notes:

- I think traffic lights (TL) is a way to categorize the values into three categories based on the quality (green = great, yellow = medium, red = bad)

In [139]:
print( "Refinement Outcome Labels: ", list( set( main_df[ "RefinementOutcome" ] )))
print("Refinement Status Labels: ", list( set( main_df[ "RefinementStatus" ] )))
crys_refdeposited = main_df["CrystalName"][main_df[ "RefinementOutcome" ] == "6 - Deposited"] 
crys_refrej =  main_df["CrystalName"][main_df[ "RefinementOutcome" ]== "7 - Analysed & Rejected" ] 
crys_reffinish = main_df["CrystalName"][main_df[ "RefinementStatus" ] == "finished"] 
crys_refpending = main_df["CrystalName"][main_df[ "RefinementOutcome" ]== "1 - Analysis Pending" ] 
crys_refrunning = main_df["CrystalName"][main_df[ "RefinementStatus" ] == "running"] 
print("Number of Refined Models Deposited: ", len(crys_refdeposited) )
print("Number of Refined Models Finished: ", len( crys_reffinish ))
print("Number of Refined Models with Analysis Pending: ", len( crys_refpending ))
print("Number of Refined Models Running: ", len( crys_refrunning ))
print("Number of Refined Models Deposited after accepted by Panddas: ", "TO BE COMPLETED!!")
# print("Number of ")
print(crys_refdeposited[:5])


Refinement Outcome Labels:  [None, '7 - Analysed & Rejected', '1 - Analysis Pending', '6 - Deposited', '3 - In Refinement']
Refinement Status Labels:  [None, 'failed', 'finished', 'running', 'CIF problem']
Number of Refined Models Deposited:  134
Number of Refined Models Finished:  906
Number of Refined Models with Analysis Pending:  210
Number of Refined Models Running:  19
Number of Refined Models Deposited after accepted by Panddas:  TO BE COMPLETED!!
276    CHIKV_MacB-x0270
287    CHIKV_MacB-x0281
295    CHIKV_MacB-x0289
300    CHIKV_MacB-x0294
301    CHIKV_MacB-x0295
Name: CrystalName, dtype: object


In [21]:
# Paths
print( "Path of latests refined compound cif:", list( main_df["RefinementCIF"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Path of latests refined pdb:", list( main_df["RefinementPDB_latest"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Path of latests refined mtz:", list( main_df["RefinementMTZ_latest"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Path of latests refined mmcif:", list( main_df["RefinementMMCIFmodel_latest"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Path of latests refined mmcif with structure factors:", list( main_df["RefinementMMCIFreflections_latest"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )
print( "Path of latests refined bound structure:", list( main_df["RefinementBoundConformation"][main_df["CrystalName"] == "CHIKV_MacB-x0270"])[0]  )


Path of latests refined compound cif: /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/compound/Z100643660.cif
Path of latests refined pdb: /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/Refine_0009/refine_9.pdb
Path of latests refined mtz: /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/Refine_0009/refine_9.mtz
Path of latests refined mmcif: None
Path of latests refined mmcif with structure factors: None
Path of latests refined bound structure: /dls/labxchem/data/lb32633/lb32633-6/processing/analysis/model_building/CHIKV_MacB-x0270/Refine_0009/refine_9.split.bound-state.pdb


Refinement Info From Panddas table

In [134]:
print( "Refinement Outcome Labels: ", list( set( pandda_df[ "RefinementOutcome" ] )))

Refinement Outcome Labels:  ['2 - PANDDA model', '3 - In Refinement']


### Task: Filter out rows that mention failed data extraction

<ins> Data was Collected </ins>
- Requirements:

    - Must have been successfully harvested
    - Must have been successfully mounted
    - Data must have succcessfully been collected 

<ins> Data was Processed by dimple </ins>


In [ ]:
collectedMainDF = main_df[ 
                    # main_df[ "HarvestStatus"] == "done"  & 
                    # main_df[ "MountingResult"][:2] == "OK" & 
                    main_df[ "DataCollectionOutcome" ] == "success"] 

# Dimple
dimpleMainDF = collectedMainDF[
                    collectedMainDF[ "DataProcessingDimpleSuccessful" ] == "TRUE" &
                     collectedMainDF[ "DimpleStatus" ] == "finished"
                                    ]

failedDimpleMainDF = collectedMainDF[
                    collectedMainDF[ "DataProcessingDimpleSuccessful" ] != "TRUE" 
]

# PanDDA
panddaMainDF = dimpleMainDF[
                    dimpleMainDF[ "DimplePANDDAwasRun" ] == "TRUE" &
                    dimpleMainDF[ "DimplePANDDAreject" ] == "FALSE"
                            ]
failedPanddaMainDF = dimpleMainDF[
                    dimpleMainDF[ "DimplePANDDAwasRun" ] == "TRUE" &
                    dimpleMainDF[ "DimplePANDDAreject" ] != "FALSE"
                            ]

hitPanddaMainDF = panddaMainDF[ 
                        panddaMainDF[ "DimplePANDDAhit"] == "TRUE"
                        ]

noHitPanddaMainDF = panddaMainDF[ 
                        panddaMainDF[ "DimplePANDDAhit"] != "TRUE"
                        ]
# Refinement
refinedMainDF = panddaMainDF[
                        panddaMainDF[ "RefinementOutcome" ] == "6 - Deposited" &
                        panddaMainDF[ "RefinementStatus" ] == "finished"
                                ]

failedRefineMainDF = panddaMainDF[
                        panddaMainDF[ "RefinementOutcome" ] == "7 - Analysed & Rejected" |
                        panddaMainDF[ "RefinementStatus" ] == "failed"
                                ]

underRefineMainDF = panddaMainDF[
                        panddaMainDF[ "RefinementOutcome" ] == "3 - In Refinement"
                                ]

pendingRefineMainDF = panddaMainDF[
                        panddaMainDF[ "RefinementOutcome" ] == "1 - Analysis Pending"
                                ]

filteredMainDF = dimpleMainDF

### Task: Obtain all the Metadata of Interest for filtered

Metadata Types:

- Protein information
    - Protein Name

- Compounds Information
    - Name of Library of Fragments Used
    - Dataset Name
    - Binding Affinity Related:
        - Compound Concentration (??maybe associated to binding affinity??) -> "CompoundConcentration"
        - Solvent Fraction () -> "SolventFraction"
        - -> "SoakTranferVol " 

- Paths of Interest


- Experimental Quality Metrics:
    

In [ ]:
# Protein
proteinNames = filteredMainDF[ "ProteinName" ]

# Compounds
datasetNames = filteredMainDF["CrystalName"]
fragLibraryNames = filteredMainDF[ "LibraryName"]


# Paths of Interest
dimplePathRefPDB = filteredMainDF[ "DimpleReferencePDB" ]
dimplePathMTZ = filteredMainDF[ "DimplePathToMTZ" ]
dimplePathPDB = filteredMainDF[ "DimplePathToPDB" ]

panddaPath = filteredMainDF[ "DimplePANDDApath" ]

refinedPathCIF = refinedMainDF[ "RefinementCIF"] # Only existsi if refinementStatus == "finished"
refinedPathPDB = refinedMainDF[ "RefinementPDB_latest"] # Only existsi if refinementStatus == "finished"
refinedPathMTZ = refinedMainDF[ "RefinementMTZ_latest"] # Only existsi if refinementStatus == "finished"
mmCIFPathModel = refinedMainDF[ "RefinementMMCIFmodel_latest"] # Only existsi if refinementStatus == "finished"
mmCIFPathSF = refinedMainDF[ "RefinementMMCIFreflections_latest"] # Only existsi if refinementStatus == "finished"


# Experimental Quality Metrics

    # Autoprocessing 
highResolution = filteredMainDF[ "DataProcessingResolutionHigh" ]
lowResolution = filteredMainDF["DataProcessingResolutionLow"]
rMerge = filteredMainDF["DataProcessingRmergeOverall"]
rMergeHigh = filteredMainDF["DataProcessingRmergeHigh"]
rMergeLow = filteredMainDF["DataProcessingRmergeLow"]
Isig = filteredMainDF["DataProcessingIsigOverall"]
IsigHigh = filteredMainDF["DataProcessingIsigHigh"]
IsigLow = filteredMainDF["DataProcessingIsigLow"]
completeness = filteredMainDF["DataProcessingCompletenessOverall"]
completenessHigh = filteredMainDF["DataProcessingCompletenessHigh"]
completenessLow = filteredMainDF["DataProcessingCompletenessLow"]
ccHalf = filteredMainDF["DataProcessingCChalfOverall"]
ccHalfHigh = filteredMainDF["DataProcessingCChalfHigh"]
ccHalfLow = filteredMainDF["DataProcessingCChalfLow"]
    # Dimple
dimpleRcrys = filteredMainDF[ "DimpleRcryst" ]
dimpleRfree = filteredMainDF[ "DimpleRfree" ]
    # Pandda

# filteredMainDF[ "" ]

---

## | **Panddas Table** |

There are lots data quality information in here:

- PANDDA_site_occupancy

- PANDDA_site_B_average

- PANDDA_site_B_ratio_residue_surroundings

- PANDDA_site_RSCC (Real Space Correlation Coefficient)

- PANDDA_site_RSR (Real Space Residuals)

- 

- <ins>Events Site Confidence</ins>: PANDDA_site_confidence (Low, Medium, High),  

Resources:

- [RSCC, RSR, and RSZO](https://www.ccp4.ac.uk/html/edstats.html)

In [138]:
print("Total number of pandas models: ", len( list( set( pandda_df[ "CrystalName" ] ))))
print("All pandda hits mentioned in the main table match all the datapoint in Pandda Table!") if set(crys_panddahit) ==  set( pandda_df[ "CrystalName" ] ) else print("Mismatch between main table and pandda table")

Total number of pandas models:  832
All pandda hits mentioned in the main table match all the datapoint in Pandda Table


<ins> Ligand Placement and Site Evaluation </ins>

"PANDDA_site_ligand_placed" - Must be manually ticked by the user, no matter whether the placing was done by the user or panddas

"PANDDA_site_viewed" - 

"PANNDA_site_interesting" - Must be manually ticked by the user as True

In [141]:
print("Total number of pandas models: ",  list( set( pandda_df[ "PANDDA_site_ligand_placed" ] )))

Total number of pandas models:  ['True', 'False']




## | **Collection Table** |

Has more information about the data collection all the autoprocessing that was used and metrics associated with each autoprocessing

There are lots data quality information in here:

- <ins>Score</ins>: DataProcessingScore, 

- <ins>UniqueReflections</ins>: DataProcessingUniqueReflectionsOverall, DataProcessingUniqueReflectionsLow, DataProcessingUniqueReflectionsHigh, 

- <ins>CChalf</ins>: DataProcessingCChalfOverall, DataProcessingCChalfLow, DataProcessingCChalfHigh

- <ins>Multiplicity</ins>: DataProcessingMultiplicityOverall, DataProcessingMultiplicityLow, DataProcessingMultiplicityHigh

- <ins>Completeness</ins>: DataProcessingCompletenessOverall, DataProcessingCompletenessLow, DataProcessingCompletenessHigh

- <ins>Processing</ins>: DataProcessingIsigOverall, DataProcessingIsigLow, DataProcessingIsigHigh 

- <ins>Rmerge</ins>: DataProcessingRmergeOverall, DataProcessingRmergeLow, DataProcessingRmergeHigh 

- <ins>Resolution</ins>: DataProcessingResolutionOverall, DataProcessingResolutionLow, DataProcessingResolutionLowInnerShell, DataProcessingResolutionHigh, DataProcessingResolutionHigh15sigma, DataProcessingResolutionHighOuterShell

<ins> Data Collection Notes:</ins>

Note that there might be more than one data collection run. This means there might be duplicates of the data, if this is not taken in consideration

Also, some crystal were reported to be soaked, harvested, mounted and exported to ispyb but data was not collected (E.g. CHIKV_MacB-x0980)

DataCollectionRun : There could be more than one run. Here the code it CrystalName_{CollectionRun}